### Import packages

In [1]:
import pandas as pd
import numpy as np
import string
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
# set seeds for reproducability
from tensorflow.keras.utils import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)
# keras module for building LSTM 
from keras.models import Model
from keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Input, Concatenate
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import keras.utils as ku

import pickle

2023-04-11 19:37:49.701532: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 19:37:51.421011: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/pardalito/miniconda3/envs/meia/lib/
2023-04-11 19:37:51.421188: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/pardalito/miniconda3/envs/meia/lib/
2023-04-11 19:37:51.421205: W tensorflow/com

### Read corpus

In [2]:
data = pd.read_csv('./nlp_data.csv')
#files = []
#for file in data['filename']:
#    with open('../corpus/'+file) as f:
#        content = f.read()
#    files.append(content)

#data = data.assign(reports=files)
data = data.values.tolist()
new_data = []
for i in range(len(data)):
    new_data.append([str(data[i][1]) + ' ' + str(data[i][2]) + ' ' + str(data[i][3]) + ' ' + str(data[i][4]) + ' ' + str(data[i][5]) + ' ' + str(data[i][6]) + ' ' + str(data[i][7]) + ' ' + str(data[i][8]) + ' ' + str(data[i][9]) + ' ' + str(data[i][10]) + ' ' + str(data[i][11]) + ' ' + str(data[i][12]) + ' ' + str(data[i][13]) + ' ' + str(data[i][14]) + ' ' + str(data[i][15]) + ' ' + str(data[i][16]) + ' ' + str(data[i][17])])

report = pd.DataFrame(new_data, columns=['reports'])
report = report['reports']
train_text, val_text = train_test_split(report, test_size=0.2)

In [3]:
train_text

35     12 August 2023 12 Ana very high 5.667 temperat...
11     17 December 2023 8 Ricardo very low 0.16 solar...
29     23 February 2023 10 Ana medium 6.0 hourly prec...
0      7 October 2023 15 Ema high 10.143 dewpoint nan...
159    11 March 2023 17 Rafael low 1.2 solar radiatio...
                             ...                        
137    8 January 2023 11 Ana medium 8.0 daily rain na...
72     4 September 2023 13 Franciso medium 3.133 temp...
140    26 December 2024 19 Diana very low 1.0 hourly ...
235    5 September 2023 18 Ricardo medium 2.867 humid...
37     5 June 2024 13 Diana medium 6.286 hourly preci...
Name: reports, Length: 204, dtype: object

In [4]:
def clean_text(txt):
    txt = "".join(t for t in txt if t not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt  

tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to a token sequence 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

def generate_padded_sequences(input_sequences,total_words):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

corpus = [clean_text(x) for x in train_text]
inp_sequences, total_words = get_sequence_of_tokens(corpus)
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences,total_words)

### Create Model

In [5]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    input_text = Input(shape=(input_len,))
    
    # Define the embedding layer for the text input
    x = Embedding(total_words, 10, input_length=input_len)(input_text)

    # Define the LSTM layer
    x = LSTM(100)(x)
    x = Dropout(0.1)(x)

    # Define the output layer
    output = Dense(total_words, activation='softmax')(x)

    # Define the model
    model = Model(inputs=input_text, outputs=output)

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

In [6]:
model = create_model(max_sequence_len, total_words)
model.summary()

test_scores = model.fit(predictors, label, epochs=50, verbose = 1)
print(test_scores)

2023-04-11 19:37:54.670405: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-11 19:37:54.684551: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-11 19:37:54.685348: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-11 19:37:54.686347: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 264)]             0         
                                                                 
 embedding (Embedding)       (None, 264, 10)           11340     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1134)              114534    
                                                                 
Total params: 170,274
Trainable params: 170,274
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50


2023-04-11 19:37:59.297747: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-04-11 19:38:00.340271: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fa5ee205840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-11 19:38:00.340331: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 with Max-Q Design, Compute Capability 7.5
2023-04-11 19:38:00.349444: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-11 19:38:00.512196: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1060/1060 [==============================] - 45s 38ms/step - loss: 5.1617
Epoch 2/50
1060/1060 [==============================] - 18s 17ms/step - loss: 4.0077
Epoch 3/50
1060/1060 [==============================] - 16s 15ms/step - loss: 3.1426
Epoch 4/50
1060/1060 [==============================] - 17s 16ms/step - loss: 2.6770
Epoch 5/50
1060/1060 [==============================] - 17s 16ms/step - loss: 2.3924
Epoch 6/50
1060/1060 [==============================] - 20s 19ms/step - loss: 2.2052
Epoch 7/50
1060/1060 [==============================] - 20s 19ms/step - loss: 2.0627
Epoch 8/50
1060/1060 [==============================] - 17s 16ms/step - loss: 1.9555
Epoch 9/50
1060/1060 [==============================] - 15s 14ms/step - loss: 1.8620
Epoch 10/50
1060/1060 [==============================] - 17s 16ms/step - loss: 1.7846
Epoch 11/50
1060/1060 [==============================] - 16s 15ms/step - loss: 1.7200
Epoch 12/50
1060/1060 [==============================] - 18s 17ms/step - l

### Serialize Model

In [7]:
with open('nlp_model.pkl', 'wb') as f:
    pickle.dump(model, f)
    f.close()
    
with open('max_sequence_len.pkl', 'wb') as f:
    pickle.dump(max_sequence_len, f)
    f.close()

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dropout
.........vars
......embedding
.........vars
............0
......input_layer
.........vars
......lstm
.........cell
............vars
...............0
...............1
...............2
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
variables.h5                                   2023-04-11 19:52:02      2070008
config.json                                    2023-04-11 19:52:02         2791
metadata.json                                  2023-04-11 19:52:02           64


### Load Model

In [8]:
with open('nlp_model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('max_sequence_len.pkl', 'rb') as f:
    max_sequence_len = pickle.load(f)

Keras model archive loading:
File Name                                             Modified             Size
variables.h5                                   2023-04-11 19:52:02      2070008
config.json                                    2023-04-11 19:52:02         2791
metadata.json                                  2023-04-11 19:52:02           64
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......dense
.........vars
............0
............1
......dropout
.........vars
......embedding
.........vars
............0
......input_layer
.........vars
......lstm
.........cell
............vars
...............0
...............1
...............2
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars


### Generate Text 

In [9]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted=np.argmax(predicted,axis=1)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if predicted == index:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [15]:
print(generate_text("3 July 2023 20 Leticia very high 1342 daily rain solar radiation significant negative impact significant negative impact 2 2 2", 200, model, max_sequence_len))

3 July 2023 20 Leticia Very High 1342 Daily Rain Solar Radiation Significant Negative Impact Significant Negative Impact 2 2 2 20 20 00 00 Dear Diana We Hope This Message Finds You Well As Your Solar Power Provider We Wanted To Provide You With Some Important Information Regarding The Solar Power Generated By Your Solar Panels On July 24Th 2024 At 1900 We Predict That Your Solar Panels Will Generate A High Amount Of Power Specifically 272 Kilo Watts This Is Due To The Positive Contribution Of The Predicted Temperature Value And Temperature Predicted Value This Means That You Can Use Your Dishwasher And Water Heater Without Exceeding The Generated Power However We Advise Against Using Your Washing Machine As It Might Exceed The Generated Power We Hope This Information Is Helpful And You To You And That You Are Able To Enjoy The Benefits Of Your Solar Panels And We Hope You Find This Report Helpful And Informative As Always If You Have Any Questions Or Concerns Please Dont Hesitate To Re